In [1]:
### GENERATE
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
TSKS33 Load data for hands-on session 5

Erik G. Larsson 2020
"""

import sys
import snap
import numpy
sys.path.append("/courses/TSKS33/ht2023/common-functions")
from save_Gephi_gexf import saveGephi_gexf
from save_Gephi_gexf import saveGephi_gexf_twocolors
from gen_stochblock import gen_stoch_block_2comm



### GENERATE

#G1=gen_stoch_block_2comm(700,300,3.0/700,3.0/300,0.1/(700+300))
G1=gen_stoch_block_2comm(700,300,10.0/700,10.0/300,0.3/(700+300))
G=snap.GetMxScc(G1)
snap.SaveEdgeList(G, "SB-small-network.txt")

G1=gen_stoch_block_2comm(10000,5000,3.0/10000,3.0/5000,0.1/(10000+5000))
G=snap.GetMxScc(G1)
snap.SaveEdgeList(G, "SB-large-network.txt")


### LOAD FROM FILES

G = snap.LoadEdgeList(snap.PUNGraph, "test-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "test-network.mat")

G = snap.LoadEdgeList(snap.PUNGraph, "karate-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "karate-network.mat")

G = snap.LoadEdgeList(snap.PUNGraph, "SB-small-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "SB-small-network.mat")

G = snap.LoadEdgeList(snap.PUNGraph, "SB-large-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "SB-large-network.mat")

#G1=gen_stoch_block_2comm(700,300,3.0/700,3.0/300,0.1/(700+300))
G1=gen_stoch_block_2comm(700,300,10.0/700,10.0/300,0.3/(700+300))
G=snap.GetMxScc(G1)
snap.SaveEdgeList(G, "SB-small-network.txt")

G1=gen_stoch_block_2comm(10000,5000,3.0/10000,3.0/5000,0.1/(10000+5000))
G=snap.GetMxScc(G1)
snap.SaveEdgeList(G, "SB-large-network.txt")


### LOAD FROM FILES

G = snap.LoadEdgeList(snap.PUNGraph, "test-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "test-network.mat")

#G = snap.LoadEdgeList(snap.PUNGraph, "karate-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "karate-network.mat")

#G = snap.LoadEdgeList(snap.PUNGraph, "SB-small-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "SB-small-network.mat")

#G = snap.LoadEdgeList(snap.PUNGraph, "SB-large-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "SB-large-network.mat")


In [2]:
G = snap.LoadEdgeList(snap.PUNGraph, "SB-large-network.txt", 0, 1)

num_nodes = G.GetNodes()
num_edges = G.GetEdges()
import scipy.sparse as sp
import numpy as np

def create_sparse_matrix(snapGraph: snap.PUNGraph):
    rows = []
    cols = []
    data = []
    
    for edge in G.Edges():
        rows.append(edge.GetSrcNId()-1)
        cols.append(edge.GetDstNId()-1)
        data.append(1) 
    coo_matrix = sp.coo_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))
    return coo_matrix



def getA(snapGraph: snap.PUNGraph):
    A = np.zeros((num_nodes, num_nodes))
    for edge in snapGraph.Edges():
        A[edge.GetSrcNId()-1][edge.GetDstNId()-1] = 1
    return A
    
def getZ(sparse_matrix: sp.coo_matrix):
    sub_matrix = np.ones((num_nodes, num_nodes))/(2*num_edges)
    result_matrix = sp.coo_matrix(sparse_matrix.toarray()-sub_matrix)
    return result_matrix

def getK(snapGraph: snap.PUNGraph):
    i = 0
    k = np.zeros(num_nodes)
    for node in snapGraph.Nodes():
        k[i] = node.GetDeg()
        i+=1
    return k
    


In [4]:
import snap_scipy
import scipy.sparse as sp
import numpy as np
G = snap.LoadEdgeList(snap.PUNGraph, "SB-large-network.txt", 0, 1)
A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)

num_nodes = G.GetNodes()
num_edges = G.GetEdges()

def power_method(A):
    x = np.random.rand(num_nodes, 1)
    for _ in range(250):
        x = (A@x) - ((k@(k.T@x))/(2*num_edges))
        x /= np.linalg.norm(x)
        eig = x.T@((A@x) - ((k@(k.T@x))/(2*num_edges)))
    return x, eig.item()

def find_eigenvector_corresponding_to_largest_eigen_value(A):
    x, lambda_hat = power_method(A)
    print("largest-magnitude eigenvalue: ", lambda_hat)
    if(lambda_hat>0):
        print("largest-positive eigenvalue: ", lambda_hat)
        print("largest-positive eigenvector: \n", x)
        return x, lambda_hat
    else:
        lambda_old = lambda_hat
        new_A = A-np.eye(num_nodes)*lambda_hat
        x, lambda_hat = power_method(new_A)
        print("largest-positive eigenvalue: ", lambda_hat+lambda_old)
        print("largest-positive eigenvector: \n", x)
        return x, lambda_hat



x, lambda_hat = find_eigenvector_corresponding_to_largest_eigen_value(A)


largest-magnitude eigenvalue:  -0.06715685581527726
largest-positive eigenvalue:  4.381872022654541
largest-positive eigenvector: 
 [[ 2.79463268e-03]
 [ 6.85124571e-04]
 [ 8.70295610e-05]
 ...
 [-2.10767038e-03]
 [-6.67654747e-03]
 [-1.42270968e-02]]


[[ 1]
 [ 1]
 [ 1]
 ...
 [-1]
 [-1]
 [-1]]


In [4]:
from save_Gephi_gexf import save_csrmatrix_Gephi_gexf_twocolors
save_csrmatrix_Gephi_gexf_twocolors(A, "large_sb.gexf", x)

In [9]:
A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)
num_edges = G.GetEdges()
kkt = np.outer(k, k)/(2*num_edges)
kkt_sparse = sp.csr_matrix(kkt)
Z = A - kkt_sparse

s = np.where(x > 0, 1, -1)

Q = (s.T@Z.toarray()@s)/(4*num_edges)

print(Q)

[[0.34460015]]


In [6]:
#eigs = np.linalg.eigvals(Z.toarray())

In [7]:
from scipy.sparse.linalg import eigsh
eigenvalues_sparse, _ = eigsh(sp.csr_matrix(Z), k=5, which='LM')

In [8]:
print(eigenvalues_sparse)

[-4.37853191 -4.09413895  4.09497283  4.24939909  4.38187269]
